# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
import json
# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials["CONSUMER_KEY"] = "P5VrFLNotO0HSNkn6Wo9mIzkG"
credentials["CONSUMER_SECRET"] = "OMrHoMvlwcVNVccZiwz3kY0seqtZ4sutrCvtTsPTr4OSuK4joA"
credentials["ACCESS_TOKEN"] = "1350133973939318785-1BzE3pShDLOzNQNdbv9Pn76IX9vvxn"
credentials["ACCESS_SECRET"] = "YFx1HaJcv2IwOjjdhQQ8XnJ0OwTQ3w43kX2lGL7u1L0RH"

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [2]:
credentials["CONSUMER_KEY"]

'P5VrFLNotO0HSNkn6Wo9mIzkG'

In [3]:
# Import the Twython class
from twython import Twython

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our Query
query_1 = {'q': 'Bitcoin',
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [4]:
import pandas as pd

# Search tweets for BitCoin
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query_1)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

,user,date,text,favorite_count
1,michael_saylor,Sat Jan 16 13:41:27 +0000 2021,"#Bitcoin is not a ponzi, fad, or bubble. It i...",8907
7,michael_saylor,Sun Jan 17 14:57:03 +0000 2021,#Bitcoin is thermal armor to replace your pape...,4500
3,tyler,Sat Jan 16 15:32:08 +0000 2021,#Bitcoin is sound money,4399
6,michael_saylor,Sun Jan 17 16:59:06 +0000 2021,"Rumor and speculation about stable coins, alt ...",3849
8,alistairmilne,Sat Jan 16 19:19:59 +0000 2021,Get some #Bitcoin first ... https://t.co/vbbGA...,2269
9,alistairmilne,Sat Jan 16 06:49:23 +0000 2021,What they think will happen if Tether collapse...,1998
2,getongab,Sun Jan 17 14:12:44 +0000 2021,Response to the Wall Street Journal about bitc...,1727
0,davidgokhshtein,Sat Jan 16 15:24:45 +0000 2021,I will give someone $50 in 24 hours. All you h...,1607
5,NickKristof,Sat Jan 16 00:40:14 +0000 2021,"Yahoo is reporting that $500,000 in bitcoin wa...",669
4,washingtonpost,Sat Jan 16 21:04:07 +0000 2021,Massive blackouts have hit Iran. The governmen...,436


In [5]:
# Create our Query for Apple Inc
query_2 = {'q': 'Apple Inc',
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [6]:
# Search tweets for Apple
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query_2)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

,user,date,text,favorite_count
1,AP,Fri Jan 15 05:46:06 +0000 2021,The U.S. government has blacklisted Chinese sm...,335
0,Inc,Sat Jan 16 19:59:30 +0000 2021,How Apple's Tim Cook controls a conversation w...,70


In [7]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [8]:
# Instantiate from our streaming class
stream = MyStreamer(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'],
                    credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='Bitcoin')

ConnectionError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

In [9]:
[ 
    credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'],
                    credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'] ]

['P5VrFLNotO0HSNkn6Wo9mIzkG',
 'OMrHoMvlwcVNVccZiwz3kY0seqtZ4sutrCvtTsPTr4OSuK4joA',
 '1350133973939318785-1BzE3pShDLOzNQNdbv9Pn76IX9vvxn',
 'YFx1HaJcv2IwOjjdhQQ8XnJ0OwTQ3w43kX2lGL7u1L0RH']